In [1]:
import numpy as np
import pandas as pd


In [4]:
loaded_data = np.load('D:\\projects\\mgr\\final\\parameters_results\\ONSBOOST__n_5__p_100__w_50__k_100__base_GaussianNB++cdi__w2_4_0,9__NC_1000__CS_100__RST_1000.npy')

In [12]:
loaded_data.reshape((9,1))

array([[0.6875    ],
       [0.79444444],
       [0.98404255],
       [0.96842105],
       [0.92319508],
       [0.69444444],
       [0.70353303],
       [0.74468085],
       [0.68235294]])